In [2]:
import numpy as np
import pandas as pd
import json


In [3]:

raw_data = json.load(open(r"C:\Users\anton\Desktop\Master-Oppgave\Kode\New_Traffic\JSON_Dupes\Volume_Dupes.json"))


*#First Attempt - Missing Both Heading Directions*

In [13]:

def flatten(item):
    return [
        {
            "id": item["data"]["trafficData"]["trafficRegistrationPoint"]["id"],
            "name": item["data"]["trafficData"]["trafficRegistrationPoint"]["name"],
            "lat": item["data"]["trafficData"]["trafficRegistrationPoint"]["location"]["coordinates"]["latLon"]["lat"],
            "lon": item["data"]["trafficData"]["trafficRegistrationPoint"]["location"]["coordinates"]["latLon"]["lon"],
            "date_from": edge["node"]["from"],
            "date_to": edge["node"]["to"],
            "volume": edge["node"]["total"]["volumeNumbers"]["volume"] if edge["node"]["total"]["volumeNumbers"] else np.nan,
            "heading1": byDirection["heading"] if byDirection["heading"] else np.nan,
            "volume_heading1": byDirection["total"]["volumeNumbers"]["volume"] if byDirection["total"]["volumeNumbers"] else np.nan
        }
        for edge in item["data"]["trafficData"]["volume"]["byDay"]["edges"]
        if edge

        for byDirection in edge["node"]["byDirection"]
        if byDirection
    ]
#"heading1": byDirection["heading"] if byDirection["heading"] else np.nan,

In [14]:
#Execute flatten function on data
data = []
for item in raw_data:
    data.extend(flatten(item))


In [15]:
#Display in dataframe
df = pd.DataFrame(data)

In [16]:
#Convert dates to YYYY-MM-DD format
df['date_from'] = pd.to_datetime(df['date_from'].str.split('T').str[0], format='%Y-%m-%d')
df['date_to'] = pd.to_datetime(df['date_to'].str.split('T').str[0], format='%Y-%m-%d')

In [18]:
df.head()

df.to_csv("volume_data.csv")

#*Attempt 2: Includes Volume for both directions, but not working, also includes heading name*

In [8]:
def flatten(item):
    return [{
            "id": item["data"]["trafficData"]["trafficRegistrationPoint"]["id"],
            "name": item["data"]["trafficData"]["trafficRegistrationPoint"]["name"],
            "lat": item["data"]["trafficData"]["trafficRegistrationPoint"]["location"]["coordinates"]["latLon"]["lat"],
            "lon": item["data"]["trafficData"]["trafficRegistrationPoint"]["location"]["coordinates"]["latLon"]["lon"],
            "date_from": edge["node"]["from"],
            "date_to": edge["node"]["to"],
            "volume": edge["node"]["total"]["volumeNumbers"]["volume"] if edge["node"]["total"]["volumeNumbers"] else np.nan,
            "heading1": byDirection["heading"] if byDirection["heading"] else np.nan,
            "volume_heading1": byDirection["total"]["volumeNumbers"]["volume"] if byDirection["total"]["volumeNumbers"] else np.nan,
            "volume_heading2": byDirection["total"]["volumeNumbers"]["volume"] if byDirection["total"]["volumeNumbers"] else np.nan,
            "heading2": byDirection["heading"] if byDirection["heading"] else np.nan,
            "coverage_heading": byDirection["total"]["coverage"]["percentage"] if byDirection["total"]["coverage"] else np.nan
        }
        for edge in item["data"]["trafficData"]["volume"]["byDay"]["edges"]
        if edge
        for byDirection in edge["node"]["byDirection"]
        if byDirection
    ]


In [9]:
#Execute flatten function
data = []
for item in raw_data:
    data.extend(flatten(item))


In [10]:
#Convert to dataframe
df = pd.DataFrame(data)

In [11]:
#Convert dates to YYYY-MM-DD format
df['date_from'] = pd.to_datetime(df['date_from'].str.split('T').str[0], format='%Y-%m-%d')
df['date_to'] = pd.to_datetime(df['date_to'].str.split('T').str[0], format='%Y-%m-%d')

In [12]:
#Display dataframe
df.head()

,id,name,lat,lon,date_from,date_to,volume,heading1,volume_heading1,volume_heading2,heading2,coverage_heading
0,03614V1740869,Tangtunnelen,63.473402,10.905876,2019-12-16,2019-12-17,12071.0,Levanger,6291.0,6291.0,Levanger,54.17
1,03614V1740869,Tangtunnelen,63.473402,10.905876,2019-12-16,2019-12-17,12071.0,Trondheim,5780.0,5780.0,Trondheim,54.17
2,03614V1740869,Tangtunnelen,63.473402,10.905876,2019-12-17,2019-12-18,17011.0,Levanger,8388.0,8388.0,Levanger,100.00
3,03614V1740869,Tangtunnelen,63.473402,10.905876,2019-12-17,2019-12-18,17011.0,Trondheim,8623.0,8623.0,Trondheim,100.00
4,03614V1740869,Tangtunnelen,63.473402,10.905876,2019-12-18,2019-12-19,16887.0,Levanger,8372.0,8372.0,Levanger,100.00


*#Attempt 3: Still not working*

In [180]:
def flatten(item):
    flattened = []
    for edge in item["data"]["trafficData"]["volume"]["byDay"]["edges"]:
        by_direction = edge["node"]["byDirection"]
        for i, direction in enumerate(by_direction):
            heading = direction["heading"]
            volume = direction["total"]["volumeNumbers"]["volume"] if direction["total"]["volumeNumbers"] else np.nan
            coverage = direction["total"]["coverage"]["percentage"] if direction["total"]["coverage"] else np.nan

            entry = {
                "id": item["data"]["trafficData"]["trafficRegistrationPoint"]["id"],
                "name": item["data"]["trafficData"]["trafficRegistrationPoint"]["name"],
                "lat": item["data"]["trafficData"]["trafficRegistrationPoint"]["location"]["coordinates"]["latLon"]["lat"],
                "lon": item["data"]["trafficData"]["trafficRegistrationPoint"]["location"]["coordinates"]["latLon"]["lon"],
                "date_from": edge["node"]["from"],
                "date_to": edge["node"]["to"],
                "volume": edge["node"]["total"]["volumeNumbers"]["volume"] if edge["node"]["total"]["volumeNumbers"] else np.nan,
                "heading": heading,
                "volume_heading": volume,
                "coverage_heading": coverage
            }

            if i == 0:
                entry["heading1"] = heading
                entry["volume_heading1"] = volume
            elif i == 1:
                entry["heading2"] = heading
                entry["volume_heading2"] = volume

            flattened.append(entry)
    return flattened


In [181]:
#Execute flatten function
data = []
for item in raw_data:
    data.extend(flatten(item))

In [182]:
#Convert to dataframe
df = pd.DataFrame(data)

,id,name,lat,lon,date_from,date_to,volume,heading,volume_heading,coverage_heading,heading1,volume_heading1,heading2,volume_heading2
0,03614V1740869,Tangtunnelen,63.473402,10.905876,2019-12-16T00:00:00+01:00,2019-12-17T00:00:00+01:00,12071.0,Levanger,6291.0,54.17,Levanger,6291.0,NaN,NaN
1,03614V1740869,Tangtunnelen,63.473402,10.905876,2019-12-16T00:00:00+01:00,2019-12-17T00:00:00+01:00,12071.0,Trondheim,5780.0,54.17,NaN,NaN,Trondheim,5780.0
2,03614V1740869,Tangtunnelen,63.473402,10.905876,2019-12-17T00:00:00+01:00,2019-12-18T00:00:00+01:00,17011.0,Levanger,8388.0,100.00,Levanger,8388.0,NaN,NaN
3,03614V1740869,Tangtunnelen,63.473402,10.905876,2019-12-17T00:00:00+01:00,2019-12-18T00:00:00+01:00,17011.0,Trondheim,8623.0,100.00,NaN,NaN,Trondheim,8623.0
4,03614V1740869,Tangtunnelen,63.473402,10.905876,2019-12-18T00:00:00+01:00,2019-12-19T00:00:00+01:00,16887.0,Levanger,8372.0,100.00,Levanger,8372.0,NaN,NaN


In [183]:
#Convert dates to YYYY-MM-DD format
df['date_from'] = pd.to_datetime(df['date_from'].str.split('T').str[0], format='%Y-%m-%d')
df['date_to'] = pd.to_datetime(df['date_to'].str.split('T').str[0], format='%Y-%m-%d')

In [184]:
#Display dataframe
df.head()

,id,name,lat,lon,date_from,date_to,volume,heading,volume_heading,coverage_heading,heading1,volume_heading1,heading2,volume_heading2
0,03614V1740869,Tangtunnelen,63.473402,10.905876,2019-12-16,2019-12-17,12071.0,Levanger,6291.0,54.17,Levanger,6291.0,NaN,NaN
1,03614V1740869,Tangtunnelen,63.473402,10.905876,2019-12-16,2019-12-17,12071.0,Trondheim,5780.0,54.17,NaN,NaN,Trondheim,5780.0
2,03614V1740869,Tangtunnelen,63.473402,10.905876,2019-12-17,2019-12-18,17011.0,Levanger,8388.0,100.00,Levanger,8388.0,NaN,NaN
3,03614V1740869,Tangtunnelen,63.473402,10.905876,2019-12-17,2019-12-18,17011.0,Trondheim,8623.0,100.00,NaN,NaN,Trondheim,8623.0
4,03614V1740869,Tangtunnelen,63.473402,10.905876,2019-12-18,2019-12-19,16887.0,Levanger,8372.0,100.00,Levanger,8372.0,NaN,NaN


*#CHATGPT SUGGESTION*



In [140]:
def flatten(item):
    return [{
            "id": item["data"]["trafficData"]["trafficRegistrationPoint"]["id"],
            "name": item["data"]["trafficData"]["trafficRegistrationPoint"]["name"],
            "lat": item["data"]["trafficData"]["trafficRegistrationPoint"]["location"]["coordinates"]["latLon"]["lat"],
            "lon": item["data"]["trafficData"]["trafficRegistrationPoint"]["location"]["coordinates"]["latLon"]["lon"],
            "date_from": edge["node"]["from"],
            "date_to": edge["node"]["to"],
            "volume": edge["node"]["total"]["volumeNumbers"]["volume"] if edge["node"]["total"]["volumeNumbers"] else np.nan,
        }
        for edge in item["data"]["trafficData"]["volume"]["byDay"]["edges"]
        if edge
        for i, byDirection in enumerate(edge["node"]["byDirection"])
        if byDirection
        for j, field in enumerate(["heading1", "volume_heading1", "heading2", "volume_heading2"])
        if i == 0 or i == 1 and j >= 2
    ] + [{
            field: byDirection["heading"] if byDirection["heading"] else np.nan,
            "volume_" + field[:-1]: byDirection["total"]["volumeNumbers"]["volume"] if byDirection["total"]["volumeNumbers"] else np.nan,
            "coverage_" + field[:-1]: byDirection["total"]["coverage"]["percentage"] if byDirection["total"]["coverage"] else np.nan
        }
        for edge in item["data"]["trafficData"]["volume"]["byDay"]["edges"]
        if edge
        for i, byDirection in enumerate(edge["node"]["byDirection"])
        if byDirection
        for j, field in enumerate(["heading1", "volume_heading1", "heading2", "volume_heading2"])
        if i == 1 and j >= 2
    ]


In [141]:
#Execute flatten function
data = []
for item in raw_data:
    data.extend(flatten(item))

In [146]:
#Convert to dataframe format
df = pd.DataFrame(data)
df.head()



,id,name,lat,lon,date_from,date_to,volume,heading2,volume_heading,coverage_heading,volume_heading2,volume_volume_heading,coverage_volume_heading
0,03614V1740869,Tangtunnelen,63.473402,10.905876,2019-12-16T00:00:00+01:00,2019-12-17T00:00:00+01:00,12071.0,NaN,NaN,NaN,NaN,NaN,NaN
1,03614V1740869,Tangtunnelen,63.473402,10.905876,2019-12-16T00:00:00+01:00,2019-12-17T00:00:00+01:00,12071.0,NaN,NaN,NaN,NaN,NaN,NaN
2,03614V1740869,Tangtunnelen,63.473402,10.905876,2019-12-16T00:00:00+01:00,2019-12-17T00:00:00+01:00,12071.0,NaN,NaN,NaN,NaN,NaN,NaN
3,03614V1740869,Tangtunnelen,63.473402,10.905876,2019-12-16T00:00:00+01:00,2019-12-17T00:00:00+01:00,12071.0,NaN,NaN,NaN,NaN,NaN,NaN
4,03614V1740869,Tangtunnelen,63.473402,10.905876,2019-12-16T00:00:00+01:00,2019-12-17T00:00:00+01:00,12071.0,NaN,NaN,NaN,NaN,NaN,NaN


In [154]:
#df.loc[df['date_from'] == '2019-02-05T00:00:00+01:00']

#Convert dates to YYYY-MM-DD format
df['date_from'] = pd.to_datetime(df['date_from'].str.split('T').str[0], format='%Y-%m-%d')
df['date_to'] = pd.to_datetime(df['date_to'].str.split('T').str[0], format='%Y-%m-%d')

,id,name,lat,lon,date_from,date_to,volume,heading2,volume_heading,coverage_heading,volume_heading2,volume_volume_heading,coverage_volume_heading
338,41379V1201935,MESNADALSARMEN,61.115044,10.456279,2019-02-05T00:00:00+01:00,2019-02-06T00:00:00+01:00,13904.0,NaN,NaN,NaN,NaN,NaN,NaN
339,41379V1201935,MESNADALSARMEN,61.115044,10.456279,2019-02-05T00:00:00+01:00,2019-02-06T00:00:00+01:00,13904.0,NaN,NaN,NaN,NaN,NaN,NaN
340,41379V1201935,MESNADALSARMEN,61.115044,10.456279,2019-02-05T00:00:00+01:00,2019-02-06T00:00:00+01:00,13904.0,NaN,NaN,NaN,NaN,NaN,NaN
341,41379V1201935,MESNADALSARMEN,61.115044,10.456279,2019-02-05T00:00:00+01:00,2019-02-06T00:00:00+01:00,13904.0,NaN,NaN,NaN,NaN,NaN,NaN
342,41379V1201935,MESNADALSARMEN,61.115044,10.456279,2019-02-05T00:00:00+01:00,2019-02-06T00:00:00+01:00,13904.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
549671,22331V248904,Eksetstranda,62.159291,6.042887,2019-02-05T00:00:00+01:00,2019-02-06T00:00:00+01:00,9843.0,NaN,NaN,NaN,NaN,NaN,NaN
549672,22331V248904,Eksetstranda,62.159291,6.042887,2019-02-05T00:00:00+01:00,2019-02-06T00:00:00+01:00,9843.0,NaN,NaN,NaN,NaN,NaN,NaN
549673,22331V248904,Eksetstranda,62.159291,6.042887,2019-02-05T00:00:00+01:00,2019-02-06T00:00:00+01:00,9843.0,NaN,NaN,NaN,NaN,NaN,NaN
549674,22331V248904,Eksetstranda,62.159291,6.042887,2019-02-05T00:00:00+01:00,2019-02-06T00:00:00+01:00,9843.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import osmnx as ox
import networkx as nx
import pandas as pd

*#Testing the coordinates*

In [ ]:
import sys
!{sys.executable} -m pip install pickle5




In [194]:


def plot_points(df):
    # Create a list of lat-lon tuples
    points = [(lat, lon) for lat, lon in zip(df['latitude'], df['longitude'])]
    
    # Convert the list of lat-lon tuples to a graph
    G = ox.graph_from_point(points, distance=1000, network_type='all')
    
    # Draw the graph
    ox.plot_graph(ox.project_graph(G))
